In [45]:
import os
import time
import csv
import glob
import datetime
import win32com.client
from datetime import datetime
import shutil
import pandas as pd
import pdfkit
from fpdf import FPDF
from invoice2data import extract_data


def save_msg(output_folder):
    """download attached emails from Ivy.

    Parameters
    ----------
    output_folder: the destination folder to save msg files

    """
    start_time = time.process_time()

    # https://stackoverflow.com/questions/61798611/get-email-list-from-outlook-using-python
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    folder = outlook.folders[1].folders['Remittance']  # this need to be changed
    # print(folder.name)

    # use the following code to get all the folder names
    # print(len(outlook.folders))
    # for i in range(len(outlook.folders)):
    #    print(outlook.folders[i].name)
    # for i in range(len(outlook.folders[1].folders)):
    #    folder = outlook.folders[1].folders[i]
    #    print((i,folder.name))

    messages = folder.Items
    for m in messages:
        attachments = m.attachments
        print(len(attachments))

        if len(attachments) <= 2:  # It is a remittance with 1 pdf attachment
            rtime = m.ReceivedTime.strftime("%d%m%Y-%H%M%S")
            m.SaveAs(path + "\\" + rtime + ".msg")

        else:  # It is email from Ivy with attachments of remittance msg
            for att in attachments:
                if "msg" in att.FileName:
                    # print(att.FileName)
                    att.saveasfile(output_folder + "\\" + str(att.index) + att.FileName)

    print("--- %.2f seconds ---" % (time.process_time() - start_time))


def save_pdf(input_folder):
    """ save attachments of msg to pdf

    Parameters
    ----------
    input_folder: the folder where msg files are saved

    """

    start_time = time.process_time()

    #output_folder = input_folder

    
    for msg in glob.glob(input_folder + '\\' + '*.msg'):
        #print(msg)
        outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
        msg_path = os.path.join(input_folder, msg)
        msg_name = os.path.splitext(msg)[0]
        #print(msg_name)
        #print(msg_path)
        m = outlook.OpenSharedItem(msg_path)

        # convert email content sent by these senders to pdf
        sender = str(m.sender)
        check = ["do-not-reply@qvalent.com", "HealtheCare PROD WF"]

        for c in check:
            if c in sender:
                body_content = c + "\n" + m.body

                x = body_content.splitlines()

                pdf = FPDF()

                pdf.add_page()

                pdf.set_font("Arial", size=12)

                # pdf.cell(200, 10, txt = c, ln = 1, align = 'C')

                for line in x:
                    pdf.cell(200, 10, txt=line, ln=1, align='C')

                pdf.output(msg_name + ".pdf")

                # download attachments and convert to pdf
        attachments = m.attachments
        for attachment in attachments:
            if "htm" in attachment.FileName:
                htm_name = msg_name + ".html"
                attachment.saveasfile(htm_name)
                pdfkit.from_file(htm_name, msg_name + ".pdf")

                os.remove(htm_name)

            elif "txt" in attachment.FileName:
                txt_name = msg_name + ".txt"

                attachment.saveasfile(txt_name)

                txt2pdf(txt_name)

                os.remove(txt_name)

            elif "PDF" in attachment.FileName:
                # print(attachment.FileName)
                attachment.saveasfile(msg_name + ".pdf")

            elif "pdf" in attachment.FileName:
                # print(attachment.FileName)
                attachment.saveasfile(msg_name + ".pdf")

    print("--- %.2f seconds ---" % (time.process_time() - start_time))

    
def txt2pdf(txt_file):
    """ save txt to pdf """
    
    pdf = FPDF()    

    # Add a page 
    pdf.add_page() 

    # set style and size of font that you want in the pdf 
    pdf.set_font("Arial", size = 12) 

    # open the text file in read mode 
    f = open(txt_file, "r") 

    # insert the texts in pdf 
    for x in f: 
        pdf.cell(200, 10, txt = x, ln = 1, align = 'C') 
    
    name = os.path.splitext(txt_file)[0]
    # save the pdf with name .pdf 
    pdf.output(name + ".pdf")
    

def to_csv(file, output_folder):
    """

    Parameters
    ----------
    file: path of pdf file used to extract data
    output_folder: folder to save extract csv

    Returns
    -------
    return new_name for rename function
    """
    try:
        data = extract_data(file)

        new_name = data['issuer'] + "_" + data['invoice_number'] + "_" + str(
            data['amount'])  # used to rename msg, pdf and csv
        # print(new_name)

        csv_name = output_folder + "\\" + new_name + ".csv"
        # print(csv_name)

        with open(csv_name, 'w', encoding='utf8', newline='') as csv_file:
            writer = csv.writer(csv_file)

            key1 = 'paid_by'
            value1 = new_name

            key2 = 'remi_number'
            value2 = data['invoice_number']

            key3 = 'Remi_sum'
            value3 = data['amount']

            # old list of dict
            lines = data['lines']

            # add key and value to the dicts in list, resulting a new list of dicts
            for i in range(len(lines)):
                lines[i][key1] = value1
                lines[i][key2] = value2
                lines[i][key3] = value3
                # print(lines[i])

            keys = lines[0].keys()
            writer.writerow(keys)
            for item in lines:
                values = item.values()
                writer.writerow(values)

        return new_name

    except Exception as e:
        print(e)
        print("Next")
        print()


def rename(input_folder, output_folder):
    """

    Parameters
    ----------
    input_folder: the folder where files to be renamed
    output_folder: the folder where renamed files to be saved

    Returns
    -------

    """
    start_time = time.process_time()

    # os.chdir(input_folder)

    # get unique file names
    name_list = os.listdir(input_folder)
    u_name = []
    for n in name_list:
        u_name.append(os.path.splitext(n)[0])

    u_name = set(u_name)
    # print(u_name)

    exts = [".pdf", ".msg"]

    for i in u_name:
        remit_file = input_folder + "\\" + i + ".pdf"

        # save remittance to csv and get unique name
        new_name = to_csv(remit_file, output_folder)
        # print(new_name)

        if new_name:  # if new_name is none (extract data failed), go to next
            for ext in exts:
                old_file = input_folder + "\\" + i + ext

                new_file = output_folder + "\\" + new_name + ext
                if os.path.isfile(new_file):
                    print('File already exists, removing.')
                    os.remove(old_file)

                else:
                    os.rename(old_file, new_file)

    print("--- %.2f seconds ---" % (time.process_time() - start_time))


def combine_data(input_folder):
    """
    
    Parameters
    ----------
    input_folder: folder where csv files to be combined

    Returns
    -------

    """

    all_filenames = [i for i in glob.glob(input_folder + '\\' + '*.csv')]

    combined_csv = pd.concat([pd.read_csv(f, dtype=str) for f in all_filenames])

    combined_csv.to_csv(".\\02-processed\\Working\\remit_data.csv", index=False, encoding='utf-8')


    
def clean(bank_statement, customer_ledger):
    """ clean bank_statement and customer_ledger

    Parameters
    ----------
    bank_statement
    customer_ledger

    Returns
    -------

    """
    df_bs = pd.read_csv(bank_statement)
    df_bs.dropna(subset=['Credits', 'Date'], inplace=True)

    df_bs['Date'] = pd.to_datetime(df_bs['Date'], format='%d%m%Y')
    df_bs.to_csv(r'C:\Users\Noah\Desktop\invoice2data\02-processed\Working\BankStatement.CSV',
                 index=False, encoding='utf-8')

    # df_cl = pd.read_excel(customer_ledger, engine='pyxlsb')
    df_cl = pd.read_csv(customer_ledger)
    df_cl.dropna(subset=['Customer Number'], inplace=True)  # delete total row
    df_cl.to_csv(r'C:\Users\Noah\Desktop\invoice2data\02-processed\Working\Customer_ledger.csv',
                 index=False, encoding='utf-8')


def filing(file, date):
    """

    Parameters
    ----------
    file: workbook.xlsx used to get all rec remittance name
    date: the date of the bank statement, used to name the folder

    Returns
    -------

    """
    df = pd.read_excel(file, sheet_name='BankRec')

    names = df['paid_by'].unique().tolist()

    data_folder = r'C:\Users\Noah\Desktop\invoice2data\02-processed'
    target_dir = os.path.join(data_folder, date)

    for name in os.listdir(data_folder):
        if not os.path.isdir(os.path.join(data_folder, name)):
            for my_name in names:
                if name.startswith(str(my_name)):
                    if not os.path.exists(target_dir):
                        os.mkdir(target_dir)
                    shutil.move(os.path.join(data_folder, name), target_dir)

    shutil.copytree(r'C:\Users\Noah\Desktop\invoice2data\02-processed\Working', os.path.join(target_dir, "Data"))


def save_processed2csv():
    start_time = time.process_time()
    path = r'C:\Users\Noah\Desktop\invoice2data\02-processed'
    os.chdir(path)

    all_filenames = [i for i in glob.glob('*.pdf')]

    # print(all_filenames)

    for f in all_filenames:
        file_path = path + "\\" + f
        save_data2csv(file_path)

    print("--- %.2f seconds ---" % (time.process_time() - start_time))



In [47]:
output_folder = r'C:\Users\Noah\Desktop\invoice2data\01-temp'
save_msg(output_folder)

3
--- 0.02 seconds ---


In [48]:
input_folder = r'C:\Users\Noah\Desktop\invoice2data\01-temp'

save_pdf(input_folder)

--- 0.02 seconds ---


In [54]:
input_folder = r'C:\Users\Noah\Desktop\invoice2data\01-temp'
output_folder = r'C:\Users\Noah\Desktop\invoice2data\02-processed'

rename(input_folder, output_folder) #会删除文件,不要用在processed文件夹

--- 2.36 seconds ---


In [55]:
path = r'C:\Users\Noah\Desktop\invoice2data\02-processed'
combine_data(path)


In [42]:
bs = r'C:\Users\Noah\Desktop\invoice2data\181120.CSV'
cl = r'C:\Users\Noah\Desktop\invoice2data\Book1.csv'

clean(bs,cl)

In [56]:
file = r'C:\Users\Noah\Desktop\invoice2data\02-processed\Working\Workbook.xlsx'
date = '18112020'  # change this everytime
filing(file,date)

In [53]:
# test new template and changed template

import csv
import os
from invoice2data import extract_data
path1 = r'C:\Users\Noah\Desktop\invoice2data\01-temp'
path2 = r'C:\Users\Noah\Desktop\invoice2data\02-processed'
path3 = r'Desktop\invoice2data\issues'
file1="2External Please find attached EFT Remittance  PEJ002717 (65.4 KB).pdf"

path = path1+"\\"+file1
data = extract_data(path)
print(data)
#print(type(data['amount']))

{'issuer': 'Mater Misericordiae', 'date': datetime.datetime(2020, 11, 18, 0, 0), 'invoice_number': 'PEJ002717', 'amount': 35300.46, 'currency': 'AUD', 'lines': [{'Inv_No': '283848', 'Amount': '2,829.75'}, {'Inv_No': '284454', 'Amount': '9,088.13'}, {'Inv_No': '284597', 'Amount': '376.00'}, {'Inv_No': '284849', 'Amount': '561.00'}, {'Inv_No': '284720', 'Amount': '198.00'}, {'Inv_No': '284700', 'Amount': '198.00'}, {'Inv_No': '284739', 'Amount': '198.00'}, {'Inv_No': '284894', 'Amount': '501.60'}, {'Inv_No': '284762', 'Amount': '198.00'}, {'Inv_No': '284789', 'Amount': '3,704.80'}, {'Inv_No': '285089', 'Amount': '308.50'}, {'Inv_No': '285076', 'Amount': '1,020.70'}, {'Inv_No': '285191', 'Amount': '4,679.40'}, {'Inv_No': '285516', 'Amount': '704.00'}, {'Inv_No': '285368', 'Amount': '3,465.00'}, {'Inv_No': '285422', 'Amount': '155.00'}, {'Inv_No': '285378', 'Amount': '465.00'}, {'Inv_No': '285460', 'Amount': '774.40'}, {'Inv_No': '285371', 'Amount': '310.00'}, {'Inv_No': '285518', 'Amount'

In [ ]:
# 拆分allocation表

import pandas as pd
path = r'C:\Users\Noah\Desktop\invoice2data\Workbook.xlsx'
df = pd.read_excel(path, sheet_name='Allocation')

names = df['Customer Number'].unique().tolist()

writer = pd.ExcelWriter("MyData.xlsx", engine='xlsxwriter')

for myname in names:
    
    mydf = df.loc[df['Customer Number']==myname]
    myname = str(myname)
    mydf.to_excel(writer, sheet_name=myname)

writer.save()
